# Feature engineering with Featuretools

# Goals

* Test viable Featuretools transformations on a reduced dataset
* Benchmark Featuretools performance
* Create a large dataset for later modeling approaches

## Setup

In [12]:
import featuretools as ft
import pandas as pd

## Data import

### Households

In [2]:
household_degrees = pd.read_excel('data/2023-06-14 Household degrees.xlsx')

In [3]:
household_degrees.dtypes

HOUSEHOLD_ID                            int64
HOUSEHOLD_RPT_NAME                     object
DEGREES_CONCAT                         object
HOUSEHOLD_SPOUSE_ID                    object
SPOUSE_DEGREES_CONCAT                  object
HOUSEHOLD_RECORD                       object
PERSON_OR_ORG                          object
HOUSEHOLD_KSM_YEAR                      int64
FIRST_KSM_GRAD_DT              datetime64[ns]
RECORD_STATUS_CODE                     object
STATUS_CHANGE_DATE             datetime64[ns]
ENTITY_DEATH_DT                datetime64[ns]
HOUSEHOLD_MASTERS_YEAR                float64
HOUSEHOLD_LAST_MASTERS_YEAR           float64
HOUSEHOLD_PROGRAM                      object
HOUSEHOLD_PROGRAM_GROUP                object
dtype: object

In [63]:
# Function to convert columns to factor or other datatype
def convert_columns_to_type(df, colnames, dtype = 'category'):
    for col in colnames:
        df[col] = df[col].astype(dtype)

# Function to convert year number to date (default: end of fiscal year date)
def year_to_fiscal_date(df, year, month = 9, day = 1):
    return pd.to_datetime(dict(year = df[year], month = month, day = day))

In [5]:
convert_columns_to_type(
    household_degrees
    , ('PERSON_OR_ORG', 'HOUSEHOLD_RECORD', 'RECORD_STATUS_CODE', 'HOUSEHOLD_PROGRAM', 'HOUSEHOLD_PROGRAM_GROUP')
    , dtype = 'category'
)

In [6]:
household_degrees[['HOUSEHOLD_ID', 'HOUSEHOLD_RECORD', 'PERSON_OR_ORG', 'HOUSEHOLD_KSM_YEAR', 'RECORD_STATUS_CODE', 'HOUSEHOLD_PROGRAM']].head()

,HOUSEHOLD_ID,HOUSEHOLD_RECORD,PERSON_OR_ORG,HOUSEHOLD_KSM_YEAR,RECORD_STATUS_CODE,HOUSEHOLD_PROGRAM
0,1858,AL,P,1932,D,FT-CB
1,1869,AL,P,1915,D,FT-CB
2,1879,AL,P,1929,D,FT-CB
3,1882,AL,P,1913,D,FT-CB
4,1900,AL,P,1911,D,FT-CB


In [61]:
household_degrees['HOUSEHOLD_KSM_DT'] = year_to_fiscal_date(df = household_degrees, year = 'HOUSEHOLD_KSM_YEAR')

In [64]:
household_degrees[['HOUSEHOLD_KSM_YEAR', 'HOUSEHOLD_KSM_DT']].head()

,HOUSEHOLD_KSM_YEAR,HOUSEHOLD_KSM_DT
0,1932,1932-09-01
1,1915,1915-09-01
2,1929,1929-09-01
3,1913,1913-09-01
4,1911,1911-09-01


### Proposals

Proposal stages: should there be one row per proposal, or one row per *stage completed* per proposal?

* Advantage of rows per stage: get a count of proposals that made it to or past a given stage for any arbitrary cutoff date.
* Advantage of one row per proposal: proposals that make it farther along don't count more than ones that only get partway.

Leaning toward one row per stage. That also allows for interesting metrics like:

$$\frac{N_\text{asked}}{N_\text{anticipated}}$$

In [78]:
proposals = pd.read_excel('data/2023-07-05 Proposals.xlsx', sheet_name = 'Select v_proposal_history_fast')

In [81]:
proposals.dtypes

PROSPECT_ID                         int64
PROSPECT_NAME                      object
PROPOSAL_TYPE                      object
PROBABILITY                        object
PROPOSAL_STATUS                    object
PROPOSAL_STATUS_GROUPED            object
START_DT_CALC              datetime64[ns]
ASK_DATE                   datetime64[ns]
CLOSE_DT_CALC              datetime64[ns]
TOTAL_ORIGINAL_ASK_AMT            float64
TOTAL_ASK_AMT                     float64
TOTAL_ANTICIPATED_AMT             float64
TOTAL_GRANTED_AMT                 float64
dtype: object

In [82]:
convert_columns_to_type(
    proposals
    , colnames = ('PROPOSAL_TYPE', 'PROPOSAL_STATUS', 'PROPOSAL_STATUS_GROUPED')
    , dtype = 'category'
)

In [96]:
proposals[['PROPOSAL_STATUS_GROUPED', 'PROPOSAL_STATUS']].head(100).value_counts()

PROPOSAL_STATUS_GROUPED  PROPOSAL_STATUS
Funded/Approved          Funded             37
Declined/Withdrawn       Withdrawn          33
                         Declined           26
Anticipated              Anticipated         3
Submitted                Submitted           1
dtype: int64

__Proposal stage rules__

Ancipated: use start date

Funded/Approved: use close date

Declined/Withdrawn: use close date

Submitted: use ask date, if available; else for funded/approved and declined/withdrawn, use close date